In [1]:
import ROOT as r
import math
import array
import os
import sys
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.backends.backend_pdf import PdfPages
import uproot
import numpy as np
from scipy.optimize import curve_fit

#______________Add include directory_______________
current_dir = os.getcwd()
include_dir = os.path.join(current_dir, 'include')
sys.path.insert(0, include_dir)
#__________________________________________________

import CONFIG
import DBPARSE
from UTILITIES import *
from SIMFITS2D import DistributionFits2D
from ROOT import gStyle, TChain, TH1F, TCanvas, TLegend

Welcome to JupyROOT 6.26/06


In [2]:

GEN2Pathp1=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass1/Pass1_data_GEN2_sbs100p_nucleon_np_model2.root"
GEN3Pathp1=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass1/Pass1_data_GEN3_sbs100p_nucleon_np_model2.root"
GEN4aPathp1=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass1/Pass1_data_GEN4a_sbs100p_nucleon_np_model2.root"
GEN4bPathp1=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass1/Pass1_data_GEN4b_sbs100p_nucleon_np_model2.root"



GEN2Pathp2=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2/Final_data_GEN2_sbs100p_nucleon_np_model2.root"
GEN3Pathp2=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2/Final_data_GEN3_sbs100p_nucleon_np_model2.root"
GEN4aPathp2=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2/Final_data_GEN4a_sbs100p_nucleon_np_model2.root"
GEN4bPathp2=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2/Final_data_GEN4b_sbs100p_nucleon_np_model2.root"

GEN3PathTiming=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2/Final_timing_GEN3_sbs100p_nucleon_np_model2.root"
GEN2PathTiming=f"/media/research/TOSHIBA EXT/GEn/outfiles/pass2/Final_timing_GEN2_sbs100p_nucleon_np_model2.root"

GEN3InSim=f"/media/research/TOSHIBA EXT/GEn/outfiles/insim/QE_Insim_GEN3_sbs100p_nucleon_np_model2.root"




In [3]:
#available branches
import ROOT as r


def list_branches(rootfile_path, tree_name="Tout"):
    # Open the ROOT file
    root_file = r.TFile.Open(rootfile_path)
    
    # Check if the file was successfully opened
    if not root_file or root_file.IsZombie():
        print(f"Error: Could not open ROOT file '{rootfile_path}'.")
        return
    
    # Access the TTree
    tree = root_file.Get(tree_name)
    
    # Check if the tree exists
    if not tree:
        print(f"Error: Tree '{tree_name}' not found in file '{rootfile_path}'.")
        root_file.Close()
        return
    
    # Get the list of branches
    branch_list = tree.GetListOfBranches()
    
    # Print all branch names
    print(f"Branches in tree '{tree_name}':")
    for branch in branch_list:
        print(branch.GetName())
    
    # Close the ROOT file
    root_file.Close()

# Example usage
list_branches(GEN3PathTiming)


Branches in tree 'Tout':
runnum
trigbits
ebeam
nu
Q2
W2
dpel
ephi
etheta
pcentral
vz
vx
vy
tr_y
tr_x
tr_th
tr_ph
xtgt
ytgt
thtgt
phtgt
thetabend
xfp
yfp
thfp
phfp
trP
trPx
trPy
trPz
ePS
xPS
eSH
xSH
ySH
eHCAL
xHCAL
yHCAL
xHCAL_exp
yHCAL_exp
dx
dy
grinch_clus_size
grinch_clus_trackindex
coin_time
new_coin_time
hcal_time
bbcal_time
BPMAx
BPMAy
Rasterx
Rastery
Raster2x
Raster2y
helicity
IHWP
pNhat_x
pNhat_y
pNhat_z
HCALtw
HODOmeantime
bb_rftime
hodo_rftime
sbs_rftime
hcal_rftime
bb_trigtime
hodo_trigtime
sbs_trigtime
hcal_trigtime
nblk
cblkid
cblke
cblktime
nbars
meantime
barid
meantot
tleft
tright
totleft
totright
vpos
pathl
npathl
pN_expected


Error in <TList::Clear>: A list is accessing an object (0x33d5ae90) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x380d6fc0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x380d73f0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x38112370) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x381126e0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x38112b50) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x38112ff0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x381133c0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x38113860) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x38113a

In [4]:
"""
#inputs hodo
config="3"
prefix="HCal_data"
detector="hodo"
#inputs hcal
config="3"
prefix="HCal_data"
detector="hcal"
"""

'\n#inputs hodo\nconfig="3"\nprefix="HCal_data"\ndetector="hodo"\n#inputs hcal\nconfig="3"\nprefix="HCal_data"\ndetector="hcal"\n'

In [4]:
#define branches
branches=["ebeam","nu","Q2","W2","dpel","ephi","etheta","pcentral","vz","vx"
          ,"vy","xtgt","ytgt","thtgt","phtgt","thetabend","xfp","yfp","thfp"
          ,"phfp","trP","trPx","trPy","trPz","ePS","xPS","eSH","xSH","ySH","eHCAL"
          ,"xHCAL","yHCAL","xHCAL_exp","yHCAL_exp","dx","dy","ngrinch_hits","xGRINCH"
          ,"yGRINCH","coin_time","hcal_time","bbcal_time","BPMAx","BPMAy","Rasterx"
          ,"Rastery","Raster2x","Raster2y","helicity","IHWP","pblkid","tdc","atime"
          ,"nblk","mag","runnum","tar","cblkid","cblkatime","cblktime","cblke","nclus"
          ,"cid","cx","cy","catime","hodoTimes","hodoIDs","hodoTOT","nbars"]
hodo_branches=["hodoTimes","hodoIDs","hodoTOT","nbars","hodoClusMean"]
hcal_branches=["cblkid","cblkatime","cblktime","cblke","nclus"
          ,"cid","cx","cy","catime"]
trigger_branches=["trigger","triggerID","NdataTriggerID"]
generic_branches=["nu","Q2","W2","vz","vx","vy","dx","dy","eSH","ePS","etheta","ephi","trP","thtgt","phtgt",
                  "coin_time","trPx","trPy","trPz","ephi","ebeam","helicity","IHWP","runnum",]
generic_branchesp2=["nu","Q2","W2","vz","vx","vy","dx","dy","eSH","ePS","etheta","ephi","trP","thtgt","phtgt",
                  "coin_time","new_coin_time","trPx","trPy","trPz","ephi","ebeam","helicity","IHWP","runnum",
                    "grinch_clus_size",
                   "grinch_clus_trackindex","HCALtw","HODOmeantime","bb_rftime","sbs_rftime"]
sim_branches=["nu","Q2","W2","vz","vx","vy","dx","dy","eSH","ePS","etheta","ephi","trP","fnucl","weight"]
insim_branches=["nu","Q2","W2","vz","vx","vy","dx","dy","eSH","ePS","etheta","trP","fnucl","weight","mc_sig",
               "mc_sigOld",
               "mc_sigPol",
               "mc_BETA",
               "mc_THETA"]


ml_branches = [
    "ebeam", "nu", "Q2", "W2", "dpel", "ephi", "etheta", "pcentral", "vz", "vx", "vy",
    "xtgt", "ytgt", "thtgt", "phtgt", "xfp", "yfp", "thfp", "phfp", "trP",
    "ePS", "eSH", "eHCAL", "xHCAL", "yHCAL", "xHCAL_exp", "yHCAL_exp",
    "dx", "dy"
]
tofcal_branches = [
    "ePS",
    "runnum",
    "tleft",
    "tright",
    "totleft",
    "totright",
    "vpos",
    "barid",
    "nbars",
    "nblk",
    "bb_rftime",
    "hodo_rftime",
    "sbs_rftime",
    "hcal_rftime",
    "bb_trigtime",
    "hodo_trigtime",
    "sbs_trigtime",
    "hcal_trigtime",
    "cblke",
    "cblkid",
    "cblktime",
    "vz",
    "xHCAL",
    "yHCAL",
    "xSH",
    "ySH",
    "pathl",
    "tr_y",
    "tr_x",
    "tr_th",
    "tr_ph",
    "meantime",
    "HCALtw",
    "W2",
    "dx",
    "dy",
    "trPx",
    "trPy",
    "trPz",
    "trP",
    "cblkid",
    "nblk",
    "cblke",
    "cblktime",
    "pN_expected",
    "npathl"
    
]


In [5]:

def ConvertToNP(branches,config,prefix,detector,File,Pass):
    # Open the ROOT file
    with uproot.open(File) as file:
        # Access the tree (replace 'Tout' with the actual name of your tree if different)
        tree = file["Tout"]

        # Get all branch names
        #branches = tree.keys()

        chunk_size = 2000000  # Define the chunk size
        total_entries = tree.num_entries  # Get the total number of entries
        print(total_entries)
        total_iterations = total_entries // chunk_size + (total_entries % chunk_size > 0)

        for j in range(total_iterations):
            print(f"Processing Chunk {j+1} out of {total_iterations}")

            start = j * chunk_size
            stop = min((j + 1) * chunk_size, total_entries)

            # Read data for this chunk
            data = tree.arrays(branches, entry_start=start, entry_stop=stop, library="np")
            
            # Save all the branches as a dictionary to .npz
            np.savez(f"/media/research/TOSHIBA EXT/GEn/NumpyData/Pass{Pass}/GEN{config}/{prefix}/{detector}{j}.npz", **data)


In [6]:
def ConvertToNPSim(branches,config,detector):
    # Open the ROOT file
    with uproot.open(f"/media/research/TOSHIBA EXT/GEn/outfiles/sim/QE_sim_GEN{config}_sbs100p_nucleon_np_model2.root") as file:
        # Access the tree (replace 'Tout' with the actual name of your tree if different)
        tree = file["Tout"]

        # Get all branch names
        #branches = tree.keys()

        chunk_size = 2000000  # Define the chunk size
        total_entries = tree.num_entries  # Get the total number of entries
        total_iterations = total_entries // chunk_size + (total_entries % chunk_size > 0)
        print(total_entries)
        for j in range(total_iterations):
            print(f"Processing Chunk {j+1} out of {total_iterations}")

            start = j * chunk_size
            stop = min((j + 1) * chunk_size, total_entries)

            # Read data for this chunk
            data = tree.arrays(branches, entry_start=start, entry_stop=stop, library="np")

            # Save all the branches as a dictionary to .npz
            np.savez(f"/media/research/TOSHIBA EXT/GEn/NumpySim/GEN{config}/{detector}{j}.npz", **data)


In [7]:
def ConvertToNPInSim(branches,config,prefix,detector,File):
    # Open the ROOT file
    with uproot.open(File) as file:
        # Access the tree (replace 'Tout' with the actual name of your tree if different)
        tree = file["Tout"]

        # Get all branch names
        #branches = tree.keys()

        chunk_size = 2000000  # Define the chunk size
        total_entries = tree.num_entries  # Get the total number of entries
        total_iterations = total_entries // chunk_size + (total_entries % chunk_size > 0)
        print(total_entries)
        for j in range(total_iterations):
            print(f"Processing Chunk {j+1} out of {total_iterations}")

            start = j * chunk_size
            stop = min((j + 1) * chunk_size, total_entries)

            # Read data for this chunk
            data = tree.arrays(branches, entry_start=start, entry_stop=stop, library="np")

            # Save all the branches as a dictionary to .npz
            np.savez(f"/media/research/TOSHIBA EXT/GEn/NumpySim/GEN{config}/{detector}{j}.npz", **data)


# TIMING

In [8]:
ConvertToNP(tofcal_branches,"3","Timing","timing",GEN3PathTiming,"2")

30506510
Processing Chunk 1 out of 16
Processing Chunk 2 out of 16
Processing Chunk 3 out of 16
Processing Chunk 4 out of 16
Processing Chunk 5 out of 16
Processing Chunk 6 out of 16
Processing Chunk 7 out of 16
Processing Chunk 8 out of 16
Processing Chunk 9 out of 16
Processing Chunk 10 out of 16
Processing Chunk 11 out of 16
Processing Chunk 12 out of 16
Processing Chunk 13 out of 16
Processing Chunk 14 out of 16
Processing Chunk 15 out of 16
Processing Chunk 16 out of 16


In [9]:
ConvertToNP(tofcal_branches,"2","Timing","timing",GEN2PathTiming,"2")

26082060
Processing Chunk 1 out of 14
Processing Chunk 2 out of 14
Processing Chunk 3 out of 14
Processing Chunk 4 out of 14
Processing Chunk 5 out of 14
Processing Chunk 6 out of 14
Processing Chunk 7 out of 14
Processing Chunk 8 out of 14
Processing Chunk 9 out of 14
Processing Chunk 10 out of 14
Processing Chunk 11 out of 14
Processing Chunk 12 out of 14
Processing Chunk 13 out of 14
Processing Chunk 14 out of 14


In [ ]:


#Hodoscope
ConvertToNP(hodo_branches,"3","HCal_data","hodo")

#HCal
ConvertToNP(hcal_branches,"3","HCal_data","hcal")

#Trigger
ConvertToNP(trigger_branches,"3","HCal_data","trigger")

# Generic Pass1

In [12]:
#Generic
ConvertToNP(generic_branches,"2","He3","generic",GEN2Pathp1,"1")

Processing Chunk 1 out of 14
Processing Chunk 2 out of 14
Processing Chunk 3 out of 14
Processing Chunk 4 out of 14
Processing Chunk 5 out of 14
Processing Chunk 6 out of 14
Processing Chunk 7 out of 14
Processing Chunk 8 out of 14
Processing Chunk 9 out of 14
Processing Chunk 10 out of 14
Processing Chunk 11 out of 14
Processing Chunk 12 out of 14
Processing Chunk 13 out of 14
Processing Chunk 14 out of 14


In [13]:
#Generic
ConvertToNP(generic_branches,"3","He3","generic",GEN3Pathp1,"1")

Processing Chunk 1 out of 13
Processing Chunk 2 out of 13
Processing Chunk 3 out of 13
Processing Chunk 4 out of 13
Processing Chunk 5 out of 13
Processing Chunk 6 out of 13
Processing Chunk 7 out of 13
Processing Chunk 8 out of 13
Processing Chunk 9 out of 13
Processing Chunk 10 out of 13
Processing Chunk 11 out of 13
Processing Chunk 12 out of 13
Processing Chunk 13 out of 13


In [15]:
#Generic
ConvertToNP(generic_branches,"4a","He3","generic",GEN4aPathp1,"1")

Processing Chunk 1 out of 5
Processing Chunk 2 out of 5
Processing Chunk 3 out of 5
Processing Chunk 4 out of 5
Processing Chunk 5 out of 5


# Generic Pass 2

In [15]:
#Generic
ConvertToNP(generic_branchesp2,"2","He3","final",GEN2Pathp2,"2")

34712274
Processing Chunk 1 out of 18
Processing Chunk 2 out of 18
Processing Chunk 3 out of 18
Processing Chunk 4 out of 18
Processing Chunk 5 out of 18
Processing Chunk 6 out of 18
Processing Chunk 7 out of 18
Processing Chunk 8 out of 18
Processing Chunk 9 out of 18
Processing Chunk 10 out of 18
Processing Chunk 11 out of 18
Processing Chunk 12 out of 18
Processing Chunk 13 out of 18
Processing Chunk 14 out of 18
Processing Chunk 15 out of 18
Processing Chunk 16 out of 18
Processing Chunk 17 out of 18
Processing Chunk 18 out of 18


In [16]:
#Generic
ConvertToNP(generic_branchesp2,"3","He3","final",GEN3Pathp2,"2")

61582929
Processing Chunk 1 out of 31
Processing Chunk 2 out of 31
Processing Chunk 3 out of 31
Processing Chunk 4 out of 31
Processing Chunk 5 out of 31
Processing Chunk 6 out of 31
Processing Chunk 7 out of 31
Processing Chunk 8 out of 31
Processing Chunk 9 out of 31
Processing Chunk 10 out of 31
Processing Chunk 11 out of 31
Processing Chunk 12 out of 31
Processing Chunk 13 out of 31
Processing Chunk 14 out of 31
Processing Chunk 15 out of 31
Processing Chunk 16 out of 31
Processing Chunk 17 out of 31
Processing Chunk 18 out of 31
Processing Chunk 19 out of 31
Processing Chunk 20 out of 31
Processing Chunk 21 out of 31
Processing Chunk 22 out of 31
Processing Chunk 23 out of 31
Processing Chunk 24 out of 31
Processing Chunk 25 out of 31
Processing Chunk 26 out of 31
Processing Chunk 27 out of 31
Processing Chunk 28 out of 31
Processing Chunk 29 out of 31
Processing Chunk 30 out of 31
Processing Chunk 31 out of 31


In [18]:
#Generic
ConvertToNP(generic_branchesp2,"4a","He3","final",GEN4aPathp2,"2")

8942070
Processing Chunk 1 out of 5
Processing Chunk 2 out of 5
Processing Chunk 3 out of 5
Processing Chunk 4 out of 5
Processing Chunk 5 out of 5


In [19]:
ConvertToNP(generic_branchesp2,"4b","He3","final",GEN4bPathp2,"2")

4934041
Processing Chunk 1 out of 3
Processing Chunk 2 out of 3
Processing Chunk 3 out of 3


In [ ]:
#MachineLearning Data/ElasticSim/InelasticSim
ConvertToNP(ml_branches,"3","HCal_data","machinelearningD")
ConvertToNPSim(ml_branches,"3","HCal_data","machinelearningES")
ConvertToNPInSim(ml_branches,"3","HCal_data","machinelearningIS")

In [15]:
#Simulation
ConvertToNPSim(sim_branches,"2","simulation")
ConvertToNPSim(sim_branches,"3","simulation")
ConvertToNPSim(sim_branches,"4","simulation")

129704
Processing Chunk 1 out of 1
264763
Processing Chunk 1 out of 1
450806
Processing Chunk 1 out of 1


In [ ]:
ConvertToNPSim(sim_branches,"2","HCal_data","simulation",3)

In [9]:
#Simulation
ConvertToNPInSim(insim_branches,"3","HCal_data","insimulation",GEN3InSim)

311751
Processing Chunk 1 out of 1
